<a href="https://colab.research.google.com/github/adryduty/computer-vision-cat-project/blob/main/Real_Time_Detection_26_June.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = red> Real Time Detection

## <font color = blue> Import the necessary modules

In [ ]:
import torch
from base64 import b64decode, b64encode
import numpy as np 
import cv2
from IPython.display import display, Javascript, Audio
from google.colab.output import eval_js
import io
from PIL import Image
import html

## <font color = blue> In the following code we build a YOLOv5s with the weights you have already stored (best.pt) 

In [ ]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', 
                       path='/content/best.pt', force_reload=True)

# model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # This is pretrained (200000 pictures)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-26 Python-3.7.13 torch-1.11.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7007428 parameters, 0 gradients
Adding AutoShape... 


## <font color = blue> Let's use the camera and the frozen weights to detect Ghiri

### The following function takes an image from the webcam as input, encodes it in numpy array format and then decode it into an OpenCV BGR image. This image will be given to the model, in order to detect Ghiri.

In [ ]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])

  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)

  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

### The following function converts an OpenCV rectangle bounding box image into base64 byte string to be overlayed on video stream. It'll be essential in order to display the red bounding box on the image

In [ ]:
def bbox_to_bytes(bbox_array):
  """
  This function converts OpenCV rectangle bounding box image 
  into base64 byte string to be overlayed on video stream

  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()

  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')

  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

### The following function uses Javascript to properly create our live video stream using the webcam as input.



In [ ]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

### The following function works in conjunction with video_stream. Basically it takes the label and the bbox (initialized to be an empty string) and returns a dictionary with different indices (create, show, capture and img). We are just interested in the values associated with the key 'img' which is the actual frame which we will pass to the js_to_image function. 

In [ ]:
def video_frame(label, bbox):
  '''
  This function takes label and bbox and returns an image in base64 format with the bounding box you pass to it.
  '''
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

### Running on Webcam Video

In [ ]:
# start streaming video from webcam
video_stream()

# label for video
label_html = 'Capturing...'

# initialze bounding box to empty
frame = ''
model.conf = 0.7
count = 0 
while True:
    js_reply = video_frame(label_html, frame)

    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])
    
    frame = Image.fromarray(frame)
    results = model(frame)


    df_results = results.pandas().xywh[0].sort_values(by = ["confidence"], ascending=False)
    
    if not df_results.empty:               #check if results df is empty or not; if it's empty no bbox needs to be drawed
      conf = df_results.iloc[0,4].astype(float)


      sound_file = "/content/beep.mp3"
      display(Audio(sound_file, autoplay=True))
      results.save()
        
      # frame = frame

<IPython.core.display.Javascript object>